<a href="https://colab.research.google.com/github/vinodsri/Applied-Gen-AI/blob/main/MCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# REAL MCP in Google Colab (In-Memory Transport - WORKS!)
# Uses FastMCP's in-memory client - perfect for notebooks
# ============================================================================

# STEP 1: Install packages
print("📦 Installing packages...\n")
!pip install -q fastmcp anthropic

print("✅ Installation complete!\n")

# STEP 2: Imports
import asyncio
import json
import warnings
from anthropic import Anthropic
from fastmcp import FastMCP, Context
from fastmcp.client import Client
from google.colab import userdata

# Suppress warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# STEP 3: Get API Key
print("🔑 Setting up Anthropic API key...")
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)
print("✅ API key configured!\n")

# ============================================================================
# STEP 4: Create MCP Server (In-Memory)
# ============================================================================
print("🛠️  Creating MCP Server...")

# Create server instance
mcp_server = FastMCP("Airbnb Listings")

# Sample data
LISTINGS = [
    {"id": "1", "title": "Cozy Studio Downtown", "price": 85, "location": "New York", "bedrooms": 1},
    {"id": "2", "title": "Spacious Loft", "price": 150, "location": "Brooklyn", "bedrooms": 2},
    {"id": "3", "title": "Park View Apartment", "price": 120, "location": "Queens", "bedrooms": 2},
    {"id": "4", "title": "Modern Studio", "price": 95, "location": "New York", "bedrooms": 1}
]

# Define MCP tools
@mcp_server.tool()
def search_listings(max_price: int = 1000, location: str = "") -> str:
    """Search for Airbnb listings by maximum price and/or location.

    Args:
        max_price: Maximum price per night (default 1000)
        location: Location to search in (e.g., "New York", "Brooklyn")

    Returns:
        JSON string with matching listings
    """
    results = LISTINGS.copy()

    # Filter by price
    if max_price < 1000:
        results = [l for l in results if l["price"] <= max_price]

    # Filter by location
    if location:
        results = [l for l in results if location.lower() in l["location"].lower()]

    return json.dumps(results, indent=2)

@mcp_server.tool()
def get_listing_details(listing_id: str) -> str:
    """Get detailed information about a specific listing.

    Args:
        listing_id: The ID of the listing (e.g., "1", "2", "3")

    Returns:
        JSON string with listing details
    """
    listing = next((l for l in LISTINGS if l["id"] == listing_id), None)

    if listing:
        # Add some extra details
        detailed = listing.copy()
        detailed["amenities"] = ["WiFi", "Kitchen", "Air Conditioning"]
        detailed["rating"] = 4.8
        return json.dumps(detailed, indent=2)
    else:
        return json.dumps({"error": "Listing not found"}, indent=2)

print("✅ MCP Server created with tools:")
print("   - search_listings")
print("   - get_listing_details\n")

# ============================================================================
# STEP 5: AI Agent with Claude + MCP
# ============================================================================

async def run_agent(query: str):
    """Run Claude agent with MCP tools"""

    print("=" * 70)
    print("🤖 CLAUDE AGENT WITH REAL MCP PROTOCOL (In-Memory)")
    print("=" * 70)
    print(f"\n👤 User: {query}\n")

    # Connect to MCP server using in-memory transport
    async with Client(mcp_server) as mcp_client:

        print("✅ Connected to MCP server via in-memory transport\n")

        # Get available tools from MCP server
        tools_list = await mcp_client.list_tools()

        print(f"📋 Available MCP tools: {[t.name for t in tools_list]}\n")

        # Convert MCP tools to Claude format
        claude_tools = []
        for tool in tools_list:
            claude_tools.append({
                "name": tool.name,
                "description": tool.description,
                "input_schema": tool.inputSchema
            })

        # Start conversation
        messages = [{"role": "user", "content": query}]

        # Agent loop - handle tool calling
        iteration = 0
        max_iterations = 5

        while iteration < max_iterations:
            iteration += 1

            # Call Claude with tools
            response = anthropic_client.messages.create(
                model="claude-3-5-sonnet-20241022",  # Current production model
                max_tokens=2048,
                tools=claude_tools,
                messages=messages
            )

            # Check if Claude wants to use a tool
            if response.stop_reason == "tool_use":
                # Extract tool use
                tool_use = next(
                    block for block in response.content
                    if block.type == "tool_use"
                )

                print(f"🔧 Claude is using MCP tool: '{tool_use.name}'")
                print(f"📥 With arguments: {json.dumps(tool_use.input, indent=2)}\n")

                # Call the REAL MCP tool via the client
                tool_result = await mcp_client.call_tool(
                    tool_use.name,
                    arguments=tool_use.input
                )

                # Extract result text
                result_text = tool_result.content[0].text

                print(f"📤 MCP Tool returned:\n{result_text}\n")

                # Add to conversation
                messages.append({"role": "assistant", "content": response.content})
                messages.append({
                    "role": "user",
                    "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": result_text
                    }]
                })

            else:
                # Got final answer
                final_text = next(
                    (block.text for block in response.content if hasattr(block, "text")),
                    "No response"
                )

                print(f"🤖 Claude's Final Answer:\n{final_text}\n")
                break

        print("=" * 70)

# ============================================================================
# STEP 6: Run Different Queries
# ============================================================================

print("🚀 Running AI Agent with MCP...\n\n")

# Test Query 1
await run_agent("Find me affordable apartments under $100 in New York")

print("\n\n")

# Test Query 2
await run_agent("Show me details for listing number 2")

print("\n\n")

# Test Query 3
await run_agent("What apartments have 2 bedrooms and cost less than $130?")

print("\n" + "=" * 70)
print("✨ SUCCESS! You just used REAL MCP Protocol!")
print("=" * 70)
print("\n📚 What happened:")
print("   ✓ Created a real MCP server with FastMCP")
print("   ✓ Connected Claude to MCP via in-memory transport")
print("   ✓ Claude discovered available MCP tools")
print("   ✓ Claude intelligently called MCP tools when needed")
print("   ✓ MCP server executed tools and returned results")
print("   ✓ Claude synthesized results into natural language")
print("\n🎯 This IS genuine MCP - using Anthropic's official SDK!")
print("   (In-memory transport instead of stdio, perfect for Colab)")

📦 Installing packages...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.7 MB/s eta 0:00:00
✅ Installation complete!

🔑 Setting up Anthropic API key...
✅ API key configured!

🛠️  Creating MCP Server...
✅ MCP Server created with tools:
   - search_listings
   - get_listing_details

🚀 Running AI Agent with MCP...


🤖 CLAUDE AGENT WITH REAL MCP PROTOCOL (In-Memory)

👤 User: Find me affordable apartments under $100 in New York

✅